In [ ]:
import os
from pathlib import Path
import time
from datetime import datetime, timedelta

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib as mpl
from matplotlib import pyplot as plt
from torch import optim
from torch.utils.data import DataLoader

from argparse import ArgumentParser


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My\ Drive/19120130_19120096_SourceCode/19120130_19120096

/content/gdrive/My Drive/19120130_19120096_SourceCode/19120130_19120096


In [ ]:
ls

argslib.py    Inference.ipynb  nuscenes_data/  README.md  train.ipynb              validation.py
experiments/  notebooks/       __pycache__/    src/       Transformer-Encoder.pdf  visualization.py


In [ ]:
!pip install pyquaternion
!pip install nuscenes-devkit
!pip install ipykernel
!python -m ipykernel install

!pip install -U gdcm

!module load deeplearning/2.10.1
!pip install pyquaternion shapely lmdb opencv-python nuscenes-devkit

!pip install Pillow
!pip install lmdb==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.1/313.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=b53b1e5796bba7eb364af4748c25317556918323c1402c91913b742242819edf
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Attempting uninstall: Shapely
    Found existing installation: shapely 2.0.2
    Uninstalling shapely-2.0.2:
      Successfully uninstalled shapely-2.0.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
ERROR: pip's dependency resolve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
Installed kernelspec python3 in /usr/local/share/jupyter/kernels/python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.6 MB/s eta 0:00:00
/bin/bash: line 1: module: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.5/306.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: lmdb
    Found existing installation: lmdb 1.4.1
    Uninstalling lmdb-1.4.1:
      Successfully uninstalled lmdb-1.4.1


In [ ]:
import os
from pathlib import Path
import time
from datetime import datetime, timedelta

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib as mpl
from matplotlib import pyplot as plt
from torch import optim
from torch.utils.data import DataLoader

import src
import src.data.collate_funcs
import src.model.network as networks
from src.data.dataloader import nuScenesMaps
from src.utils import MetricDict

from src.data.dataloader import nuScenesMaps, read_split
from argslib import parse_args, init


In [ ]:

def load_checkpoint(args, model, ckpt_name=None, ckpt_epoch=None):
    model_dir = Path(args.savedir) / args.name
    if not ckpt_name:
        if ckpt_epoch:
            ckpt_name = f'checkpoint-{str(ckpt_epoch).zfill(4)}.pth.gz'
        else:
            checkpt_fn = sorted(
                [
                    f
                    for f in os.listdir(str(model_dir))
                    if os.path.isfile(os.path.join(model_dir, f)) and ".pth.gz" in f
                ]
            )
            ckpt_name = checkpt_fn[-1]
    ckpt_fname = model_dir / ckpt_name
    print(f'Loading checkpoint {ckpt_name}')
    assert (ckpt_fname.exists())

    model_pth = os.path.join(model_dir, str(ckpt_fname))
    ckpt = torch.load(model_pth)
    model.load_state_dict(ckpt["model"])

In [ ]:

def get_model(args):
    # Build model
    model = networks.__dict__[args.model_name](
        num_classes=len(args.pred_classes_nusc),
        frontend=args.frontend,
        grid_res=args.grid_res,
        pretrained=args.pretrained,
        img_dims=args.desired_image_size,
        z_range=args.z_intervals,
        h_cropped=args.cropped_height,
        dla_norm=args.dla_norm,
        additions_BEVT_linear=args.bevt_linear_additions,
        additions_BEVT_conv=args.bevt_conv_additions,
        dla_l1_n_channels=args.dla_l1_nchannels,
        n_enc_layers=args.n_enc_layers,
        n_dec_layers=args.n_dec_layers,
    )

    device = torch.device("cuda")
    model = nn.DataParallel(model)
    _ = model.to(device)

    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True

    return model

In [ ]:

def time_val_epoch(N, fname=None):
    i = 0

    # Print summary
    while True:
        start_time = time.perf_counter()
        yield
        i = i + 1
        batch_time = (time.perf_counter() - start_time)
        eta = (N - i) * batch_time

        s = "[Val: {:4d}/{:4d}] batch_time: {:.2f}s eta: {:s}".format(
            i, N, batch_time, str(timedelta(seconds=int(eta)))
        )

        if fname:
            with open(fname, "a") as fp:
                fp.write(s + '\n')
        print(s)

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    res_100 = None
    time_epoch_gen = time_val_epoch(N=len(dataloader))
    next(time_epoch_gen)
    for i, ((image, calib, grid2d), (cls_map, vis_mask)) in enumerate(dataloader):
        with torch.no_grad():
            pred_ms = model(image, calib, grid2d)

            # Upsample the largest prediction to 200x200
            pred_200x200 = F.interpolate(
                pred_ms[0], size=(200, 200), mode="bilinear"
            )
            pred_ms = [pred_200x200, *pred_ms]
            pred_ms_cpu = [pred.detach().cpu() for pred in pred_ms]

            if res_100 is not None:
                res_100 = torch.cat((res_100, pred_ms_cpu[1]))
            else:
                res_100 = torch.cat((pred_ms_cpu[1],))

        next(time_epoch_gen)

    return res_100

In [ ]:

def collate_nusc_s(batch):
    """
    Collate fuction for:
        - NuScenes
        - Singe image input models
        - Ground truths with occluded regions masked out
    """

    images, cls_maps, vis_masks, calibs, grids2d = zip(*batch)

    # Stack images and calibration matrices along the batch dimension
    images = torch.stack(images)
    cls_maps = torch.stack(cls_maps)
    vis_masks = torch.stack(vis_masks)
    calibs = torch.stack(calibs)
    grids2d = torch.stack(grids2d)

    return (images, calibs, grids2d), (cls_maps, vis_masks)

In [ ]:
# def main():
args = parse_args(notebook=False)
init(args)
print("===================================================")

val_data = nuScenesMaps(
    root=args.root,
    split=args.val_split,
    grid_size=args.grid_size,
    grid_res=args.grid_res,
    classes=args.load_classes_nusc,
    dataset_size=args.data_size,
    desired_image_size=args.desired_image_size,
    mini=True,
    gt_out_size=(100, 100),
)
val_loader = DataLoader(
    val_data,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=1,
    collate_fn=collate_nusc_s,
    drop_last=True,
    pin_memory=True
)

ckpt_epoch = 22
# for ckpt_epoch in list(range(31, 41)):
model = get_model(args)
load_checkpoint(args, model, ckpt_epoch=ckpt_epoch)
res_100 = evaluate(model, val_loader)

experiment_dir = Path(args.savedir) / args.name
results_dir = experiment_dir / 'inference_results'
results_dir.mkdir(exist_ok=True)
torch.save(res_100, results_dir / f'ckpt-{ckpt_epoch}-full22-pred-100x100.pt')


tensor([66., 25.,  0., -0.])
Loading checkpoint checkpoint-0022.pth.gz


RuntimeError: ignored

In [ ]:
for i, ((image, calib, grid2d)) in enumerate(val_loader):
    print("aaaaa")
    print(i)

In [ ]:
args.root

In [ ]:
print(res_100.shape)

In [ ]:
if __name__=='__main__':
    main()


In [ ]:
res_100

In [ ]:
ls

In [ ]:
import os
import datetime

file_path = 'checkpoint-0012.pth/'
#get created time
n = os.path.getctime(file_path)
#convert epoch to a timestamp
timestamp = datetime.datetime.fromtimestamp(n).strftime('%c')
print(timestamp)